# ML Challenge

ESCHYLLE Annaly

GONIN Tom

MOHAMED Marwa

PESON Derhen

PETRI-GUASCO Jean-Georges

### Import des modules

In [ ]:
pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Librairies : 

# pandas 
import pandas as pd

# numpy
import numpy as np

# time
import time

# lazypredict
from lazypredict.Supervised import LazyClassifier

# sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



In [ ]:
#récupération des fichiers 
!git clone https://github.com/marwa-mohamed-dev/test_MLChallenge
%cd test_MLChallenge
#supprimer dossier
#%cd ..
#%rm -r test_MLChallenge

Cloning into 'test_MLChallenge'...
remote: Enumerating objects: 1642, done.
remote: Counting objects: 100% (335/335), done.
remote: Compressing objects: 100% (332/332), done.
remote: Total 1642 (delta 5), reused 331 (delta 3), pack-reused 1307
Receiving objects: 100% (1642/1642), 89.90 MiB | 15.51 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (1611/1611), done.
/content/test_MLChallenge


## Description des données :

In [ ]:
data = pd.read_csv("training_data.csv")
data

,id,target,targetName,subject,x_0,x_1,x_2,x_3,x_4,x_5,...,y_58,y_59,y_60,y_61,y_62,y_63,y_64,y_65,y_66,y_67
0,08_213_0699,8,Fearful,213,439.30,437.60,441.80,450.70,467.20,494.60,...,551.90,544.30,527.10,522.40,524.40,523.30,529.70,527.20,528.80,526.60
1,03_390_3151,3,Sad,390,382.30,383.80,387.20,392.90,406.80,432.20,...,564.70,556.00,538.40,532.50,535.10,532.90,539.40,552.40,555.40,552.60
2,06_290_8569,6,Suprised,290,335.40,334.50,340.40,351.20,367.40,391.70,...,550.70,544.10,524.00,511.30,510.80,509.60,519.80,525.30,526.70,526.00
3,08_201_9378,8,Fearful,201,413.50,413.40,418.70,429.80,447.90,474.60,...,589.40,581.60,562.90,556.80,558.90,557.50,565.70,568.60,570.10,567.20
4,01_346_4226,1,neutral,346,400.30,401.80,407.00,414.70,428.20,450.90,...,549.20,542.70,527.60,528.70,529.60,527.30,525.30,528.20,529.60,528.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,08_362_0441,8,Fearful,362,375.00,375.40,379.70,386.00,396.50,414.90,...,555.70,545.80,524.00,515.10,516.20,514.80,523.20,535.90,537.30,535.40
1282,06_322_1917,6,Suprised,322,394.30,393.80,400.10,412.50,429.50,453.20,...,602.90,595.40,575.80,569.20,569.80,568.50,573.10,580.80,582.40,580.90
1283,06_256_4937,6,Suprised,256,423.80,424.80,433.70,446.70,463.70,487.70,...,593.10,587.60,571.10,558.10,557.40,556.20,565.00,567.30,568.70,568.40
1284,01_272_6686,1,neutral,272,352.80,358.40,366.50,377.70,396.70,426.20,...,513.90,512.00,500.00,490.50,489.50,485.70,488.20,487.20,491.10,492.00


In [ ]:
data.shape

(1286, 140)

In [ ]:
data.columns

Index(['id', 'target', 'targetName', 'subject', 'x_0', 'x_1', 'x_2', 'x_3',
       'x_4', 'x_5',
       ...
       'y_58', 'y_59', 'y_60', 'y_61', 'y_62', 'y_63', 'y_64', 'y_65', 'y_66',
       'y_67'],
      dtype='object', length=140)

Nous vérifions si notre jeu de données est composé de valeurs manquantes :

In [ ]:
data.isna().sum().sum()

0

Nous allons regarder maintenant la répartition de nos données au niveau de notre target :

In [ ]:
data['target'].value_counts() / len(data)

5   0.15
3   0.15
7   0.15
6   0.14
8   0.14
1   0.13
2   0.13
Name: target, dtype: float64

Chaque sentiment représente 13% à 15% de notre jeu de données. Nous pouvons affirmer que nous n'avons aucun problème de répartitions des classes. 

## Analyse des données :

Matrice de corrélation :

In [ ]:
corr_matrix = data.corr()
corr_matrix

,target,subject,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,y_58,y_59,y_60,y_61,y_62,y_63,y_64,y_65,y_66,y_67
target,1.00,-0.01,0.01,0.01,0.02,0.03,0.03,0.02,0.02,0.01,...,-0.01,-0.00,-0.00,-0.03,-0.04,-0.03,-0.01,-0.01,-0.01,-0.01
subject,-0.01,1.00,0.11,0.12,0.11,0.11,0.09,0.07,0.05,0.02,...,-0.13,-0.13,-0.11,-0.11,-0.11,-0.11,-0.12,-0.12,-0.12,-0.12
x_0,0.01,0.11,1.00,1.00,1.00,0.99,0.99,0.99,0.98,0.97,...,0.63,0.63,0.62,0.62,0.62,0.62,0.63,0.63,0.63,0.63
x_1,0.01,0.12,1.00,1.00,1.00,1.00,0.99,0.99,0.98,0.97,...,0.63,0.63,0.63,0.62,0.62,0.62,0.63,0.64,0.64,0.64
x_2,0.02,0.11,1.00,1.00,1.00,1.00,1.00,0.99,0.99,0.98,...,0.64,0.64,0.63,0.63,0.63,0.63,0.64,0.64,0.64,0.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
y_63,-0.03,-0.11,0.62,0.62,0.63,0.64,0.66,0.68,0.71,0.73,...,0.98,0.99,1.00,1.00,1.00,1.00,1.00,0.99,0.99,0.99
y_64,-0.01,-0.12,0.63,0.63,0.64,0.65,0.66,0.69,0.71,0.74,...,0.99,0.99,1.00,1.00,1.00,1.00,1.00,0.99,0.99,0.99
y_65,-0.01,-0.12,0.63,0.64,0.64,0.65,0.67,0.69,0.72,0.75,...,1.00,1.00,0.99,0.99,0.99,0.99,0.99,1.00,1.00,1.00
y_66,-0.01,-0.12,0.63,0.64,0.64,0.65,0.67,0.69,0.72,0.75,...,1.00,1.00,0.99,0.99,0.99,0.99,0.99,1.00,1.00,1.00


In [ ]:
corr_list = corr_matrix.unstack().sort_values(ascending=True)
corr_list = corr_list[corr_list != 1.0]
print(corr_list)

subject  y_8       -0.18
y_8      subject   -0.18
y_9      subject   -0.18
subject  y_9       -0.18
         y_7       -0.17
                    ... 
x_61     x_50       1.00
x_52     x_63       1.00
x_63     x_52       1.00
x_51     x_62       1.00
x_62     x_51       1.00
Length: 18906, dtype: float64


## Modèle d'entrainement pour la reconnaisse d'émotion à partir des données du fichier Excel training_data.csv

### Recherche de modèle ML pour exploiter les données du fichier training_data.csv

In [ ]:
y = data.iloc[:,1]
x = data.iloc[:,4:140]


In [ ]:
# train and test splitting
X_train, X_test, y_train, y_test = model_selection.train_test_split(x, y, train_size=0.7)


Nous avions d'abord souhaiter tester sur nos données, un modèle simple que nous connaissons, le KNN.

In [ ]:
# define classifier
NN1 = KNeighborsClassifier(8)

NN1.fit(X_train, y_train)
print("Accuracy: {0:.5}%".format(NN1.score(X_test, y_test)*100))

Accuracy: 32.902%


Le résultat n'étant pas très significatif, nous avons décidé de rechercher un autre modèle de ML.

Nous avons trouvé la librairie Lazypredict qui compare différents modèles et effectue une préduction et un calcul de l'accuracy et du temps d'execution. 

In [ ]:
# Fit all models with lazyPredict
clf = LazyClassifier(predictions=True)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [00:25<00:00,  1.13it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
CalibratedClassifierCV,0.77,0.77,None,0.77,4.62
LinearSVC,0.76,0.76,None,0.76,1.42
LogisticRegression,0.75,0.75,None,0.75,0.35
RidgeClassifier,0.70,0.71,None,0.69,0.07
LinearDiscriminantAnalysis,0.70,0.70,None,0.70,0.10
RidgeClassifierCV,0.69,0.70,None,0.69,0.14
SGDClassifier,0.66,0.66,None,0.67,0.19
Perceptron,0.60,0.60,None,0.60,0.17
NuSVC,0.59,0.59,None,0.60,0.39


Selon la sélection des données de test et d'entrainement par la fonction train_test_split, le résultat peut différer. Toutefois, même si RidgeClassifierCV n'est pas toujours en tête, il reste très souvant parmi les 5 meilleurs modèles avec une différence d'accuracy minime et un meilleur temps d'exécution. Nous avons donc choisi d'utiliser le modèle RidgeClassifierCV.

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(x, y, train_size=0.7)

RidgeClass = RidgeClassifierCV()
RidgeClass.fit(X_train, y_train)
print("Accuracy: {0:.5}%".format(RidgeClass.score(X_test, y_test)*100))

Accuracy: 72.539%


Etant donné que selon la séparation entre les données de test et d'entrainement, l'accuracy change, nous avons calculer une accuracy moyenne et avons obtenu le résultat d'environ 71,5% d'accuracy :

In [ ]:
results = [] # liste pour stocker les résultats
times = [] # liste pour stocker les temps d'exécution

for size in range(1, 100):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(x, y, train_size=0.7)
    #start
    start = time.time()
    RidgeClass = RidgeClassifierCV()
    RidgeClass.fit(X_train, y_train)
    #end
    end = time.time()
    #print(end - start)
    times.append(end - start) # Ajouter le temps d'exécution à la liste
    accuracy = RidgeClass.score(X_test, y_test)
    results.append(accuracy) # Ajouter le résultat à la liste

mean_accuracy = np.mean(results) # Calculer la moyenne des résultats
mean_time = np.mean(times) # Calculer la moyenne des temps d'exécution
maximum_accuracy = np.max(results)
print("Moyenne des Accuracy : {0:.5}%".format(mean_accuracy*100)) # Afficher la moyenne des résultats
print("Moyenne des temps d'exécution : {0:.5}s".format(mean_time)) # Afficher la moyenne des temps d'exécution
print("Accuracy maximal obtenu : {0:.5}%".format(maximum_accuracy*100))

Moyenne des Accuracy : 71.448%
Moyenne des temps d'exécution : 0.070349s
Accuracy maximal obtenu : 77.202%


### Traitement des données : calcul des distances

Afin d'améliorer le score obtenu, il faut rendre les données plus exploitable. Nous avons pensé à effectuer un calcul de distance à partir des coordonnées des points. 

In [ ]:
  # calcul de la distance
def calculeDistance(x) :
    x_0 = 0
    y_0 = 0

    x_x = x.loc[:, "x_0":"x_67"]
    x_y = x.loc[:, "y_0":"y_67"]
    row = len(x)
    column = 68
    dataDistance = [[0] * column for _ in range(row)]
    for j in range(column):
        for i in range(row):
            dataDistance[i][j] = np.sqrt((x_0 - x_x.iloc[i, j]) ** 2 + (y_0 - x_y.iloc[i, j]) ** 2)
    x_dist = pd.DataFrame(dataDistance)
    return x_dist
x_dist = calculeDistance(x)
x_dist


,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
0,555.81,587.57,627.00,671.00,719.29,770.12,816.70,860.39,894.51,921.02,...,803.09,780.65,761.22,785.57,801.32,816.30,848.93,817.89,803.15,787.33
1,543.92,578.24,613.75,650.55,690.54,732.90,775.72,817.08,851.50,877.57,...,777.61,754.54,731.03,755.41,773.25,791.11,820.74,804.59,787.22,769.15
2,495.48,526.05,563.21,603.76,646.53,690.95,732.35,770.78,798.17,817.12,...,731.51,713.98,694.04,701.99,710.04,717.93,744.21,729.58,721.77,712.77
3,565.53,598.87,635.90,678.34,723.27,772.64,820.62,866.51,901.70,926.90,...,827.20,805.37,782.19,805.45,820.20,833.57,863.05,840.66,827.73,812.39
4,549.61,580.67,615.03,651.14,688.07,727.33,765.23,801.96,831.94,855.28,...,768.46,748.78,728.40,754.94,766.27,776.35,802.36,776.08,765.55,754.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,531.25,559.75,591.62,624.72,661.82,701.78,742.51,783.47,815.18,840.80,...,740.55,718.19,695.68,710.59,723.68,737.14,780.52,752.73,739.36,725.64
1282,572.88,602.89,637.72,677.14,718.42,761.86,802.30,840.61,868.92,889.16,...,803.80,785.59,765.75,778.63,788.69,797.78,823.60,807.15,798.04,787.23
1283,582.28,616.73,658.34,701.89,746.80,791.43,832.76,872.92,903.23,926.61,...,830.26,813.09,796.98,805.05,813.63,821.96,848.85,829.88,821.48,812.01
1284,515.75,550.54,587.76,625.19,662.90,699.69,732.53,763.98,792.84,818.40,...,727.54,710.49,691.24,709.90,722.93,735.94,766.26,737.45,724.31,711.30


Nous avons alors recalculer le score moyen obtenu avec notre modèle (RidgeClassifierCV). Cependant, cela a très faiblement amélioré le score. il est passé de 71.5% à 72.2% :

In [ ]:

results = [] # liste pour stocker les résultats
times = [] # liste pour stocker les temps d'exécution

for size in range(1, 100):
    X_train_dist, X_test_dist, y_train_dist, y_test_dist = model_selection.train_test_split(x_dist, y, train_size=0.7)
    #start
    start = time.time()
    RidgeClass = RidgeClassifierCV()
    RidgeClass.fit(X_train_dist, y_train_dist)
    #end
    end = time.time()
    #print(end - start)
    times.append(end - start) # Ajouter le temps d'exécution à la liste
    accuracy = RidgeClass.score(X_test_dist, y_test_dist)
    results.append(accuracy) # Ajouter le résultat à la liste

mean_accuracy = np.mean(results) # Calculer la moyenne des résultats
mean_time = np.mean(times) # Calculer la moyenne des temps d'exécution
maximum_accuracy = np.max(results)
print("Moyenne des Accuracy : {0:.5}%".format(mean_accuracy*100)) # Afficher la moyenne des résultats
print("Moyenne des temps d'exécution : {0:.5}s".format(mean_time)) # Afficher la moyenne des temps d'exécution
print("Accuracy maximal obtenu : {0:.5}%".format(maximum_accuracy*100))

Moyenne des Accuracy : 72.214%
Moyenne des temps d'exécution : 0.034311s
Accuracy maximal obtenu : 76.943%


Nous avons alors relancer lazypredict afin de potentiellement trouvé un modèle plus performant mais il n'y avait pas d'amélioration importante du score.

In [ ]:
# train and test splitting
X_train_dist, X_test_dist, y_train_dist, y_test_dist = model_selection.train_test_split(x_dist, y, train_size=0.7)


In [ ]:
# Fit all models
clf = LazyClassifier(predictions=True)
models, predictions = clf.fit(X_train_dist, X_test_dist, y_train_dist, y_test_dist)
models

100%|██████████| 29/29 [00:14<00:00,  2.04it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.75,0.76,None,0.76,0.07
RidgeClassifierCV,0.74,0.74,None,0.73,0.06
LinearSVC,0.72,0.72,None,0.71,0.82
CalibratedClassifierCV,0.72,0.72,None,0.71,4.11
RidgeClassifier,0.69,0.69,None,0.67,0.03
NuSVC,0.69,0.69,None,0.69,0.17
LogisticRegression,0.64,0.64,None,0.63,0.14
Perceptron,0.55,0.54,None,0.52,0.03
LGBMClassifier,0.48,0.49,None,0.47,5.52


In [ ]:

results = [] # liste pour stocker les résultats
times = [] # liste pour stocker les temps d'exécution

for size in range(1, 100):
    X_train_dist, X_test_dist, y_train_dist, y_test_dist = model_selection.train_test_split(x_dist, y, train_size=0.7)
    #start
    start = time.time()
    model = LinearDiscriminantAnalysis()
    model.fit(X_train_dist, y_train_dist)
    #end
    end = time.time()
    #print(end - start)
    times.append(end - start) # Ajouter le temps d'exécution à la liste
    accuracy = model.score(X_test_dist, y_test_dist)
    results.append(accuracy) # Ajouter le résultat à la liste

mean_accuracy = np.mean(results) # Calculer la moyenne des résultats
mean_time = np.mean(times) # Calculer la moyenne des temps d'exécution
maximum_accuracy = np.max(results)
print("Moyenne des Accuracy : {0:.5}%".format(mean_accuracy*100)) # Afficher la moyenne des résultats
print("Moyenne des temps d'exécution : {0:.5}s".format(mean_time)) # Afficher la moyenne des temps d'exécution
print("Accuracy maximal obtenu : {0:.5}%".format(maximum_accuracy*100))

Moyenne des Accuracy : 73.73%
Moyenne des temps d'exécution : 0.026395s
Accuracy maximal obtenu : 80.57%


Nous pouvons voir que nous atteignons 80% d'accuracy maximal ! 
Nous allons essayer d'optimiser ce modèle en utilisant un GridSearch qui va nous permettre de trouver les hyperparamètres parfaits pour notre modèle

In [ ]:
# train and test splitting
X_train_dist, X_test_dist, y_train_dist, y_test_dist = model_selection.train_test_split(x_dist, y, train_size=0.7)

# define the classifier :
model_Liner_discri = LinearDiscriminantAnalysis()

# define the hyper-parameters candidates
param_grid = {
    'solver': ['svd', 'lsqr', 'eigen'],
    'shrinkage': [None, 'auto', 0.1, 0.5, 0.9],
    'tol': [1e-4, 1e-3, 1e-2],
    'n_components': [None, 2, 5, 10],
    'store_covariance': [True, False],
}

# perform grid search
CV_model_Liner_discri = GridSearchCV(
    estimator=model_Liner_discri, param_grid=param_grid, cv=6, 
    scoring='accuracy')
CV_model_Liner_discri.fit(X_train_dist, y_train_dist)

GridSearchCV(cv=6, estimator=LinearDiscriminantAnalysis(),
             param_grid={'n_components': [None, 2, 5, 10],
                         'shrinkage': [None, 'auto', 0.1, 0.5, 0.9],
                         'solver': ['svd', 'lsqr', 'eigen'],
                         'store_covariance': [True, False],
                         'tol': [0.0001, 0.001, 0.01]},
             scoring='accuracy')

In [ ]:
print("Meilleurs paramètres : ", CV_model_Liner_discri.best_params_)
print("Meilleur score : ", CV_model_Liner_discri.best_score_)

Meilleurs paramètres :  {'n_components': None, 'shrinkage': None, 'solver': 'svd', 'store_covariance': True, 'tol': 0.001}
Meilleur score :  0.7388888888888889


In [ ]:
LinearDiscriminantAnalyse = LinearDiscriminantAnalysis(n_components=None, shrinkage=None, solver='svd', store_covariance= True, tol= 0.001)
LinearDiscriminantAnalyse.fit(X_train_dist, y_train_dist)

LinearDiscriminantAnalysis(store_covariance=True, tol=0.001)

In [ ]:
pred=LinearDiscriminantAnalyse.predict(X_test_dist)
print("Accuracy for LinearDiscriminantAnalyse data: ",accuracy_score(y_test_dist,pred))

Accuracy for Classifier Ridge on CV data:  0.7409326424870466


In [ ]:
results = [] # liste pour stocker les résultats
times = [] # liste pour stocker les temps d'exécution

for size in range(1, 100):
    X_train_dist, X_test_dist, y_train_dist, y_test_dist = model_selection.train_test_split(x_dist, y, train_size=0.7)
    #start
    start = time.time()
    LinearDiscriminantAnalyse = LinearDiscriminantAnalysis(n_components=6, shrinkage=None, solver='svd', store_covariance= True, tol= 0.01)
    LinearDiscriminantAnalyse.fit(X_train_dist, y_train_dist)
    #end
    end = time.time()
    #print(end - start)
    times.append(end - start) # Ajouter le temps d'exécution à la liste
    accuracy =  LinearDiscriminantAnalyse.score(X_test_dist, y_test_dist)
    results.append(accuracy) # Ajouter le résultat à la liste

mean_accuracy = np.mean(results) # Calculer la moyenne des résultats
mean_time = np.mean(times) # Calculer la moyenne des temps d'exécution
maximum_accuracy = np.max(results)
print("Moyenne des Accuracy : {0:.5}%".format(mean_accuracy*100)) # Afficher la moyenne des résultats
print("Moyenne des temps d'exécution : {0:.5}s".format(mean_time)) # Afficher la moyenne des temps d'exécution
print("Accuracy maximal obtenu : {0:.5}%".format(maximum_accuracy*100))

Moyenne des Accuracy : 74.551%
Moyenne des temps d'exécution : 0.027395s
Accuracy maximal obtenu : 79.534%


Matrice de confusion :

In [ ]:
from sklearn.metrics import confusion_matrix


confusion = confusion_matrix(y_test_dist, pred)
confusion

array([[12,  5,  9,  7,  6,  2,  5],
       [ 8,  6, 10, 12,  5,  8,  3],
       [12,  6,  6, 10, 10,  8,  5],
       [ 9,  9,  9,  9,  4,  7,  8],
       [11,  5,  9,  7,  4, 10,  8],
       [14, 10,  9, 10,  9, 11,  3],
       [13,  6,  4,  8,  7,  9,  9]])

Cette matrice permet de visualiser les résultats de la classification pour chaque classe dans une matrice, où les colonnes représentent les classes prédites et les lignes représentent les classes réelles.

La diagonale principale représente le nombre de bonnes prédictions par classe.

In [ ]:
#prediction

predictions = model.predict(X_test_dist)
predictions = pd.DataFrame(predictions)
predictions

,0
0,8
1,7
2,7
3,8
4,5
...,...
381,1
382,1
383,7
384,6


## Prédiction avec les données de tests

In [ ]:
data_test = pd.read_csv("data_test.csv")
x_test = data.iloc[:,4:140]
y_test = data.iloc[:,1]
test = calculeDistance(x_test)
predictions = model.predict(test)
predictions = pd.DataFrame(predictions, columns=["label"])
result = data_test.assign(label=predictions)
predictions = predictions.assign(target = y_test)
predictions
data_test

,label
0,6
1,7
2,6
3,3
4,1
...,...
1281,8
1282,6
1283,6
1284,1


In [ ]:
result.to_csv("result.csv")
predictions.to_csv("predictions.csv")

## ENTRAINEMENT DES IMAGES : UTILISATION D'UN CNN

In [4]:
#%cd test_MLChallenge/

/content/test_MLChallenge


In [5]:
import tensorflow as tf
import scipy
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.utils import load_img, img_to_array
from keras.callbacks import ReduceLROnPlateau


# Define some hyperparameters
img_height = 128
img_width = 128
batch_size = 32
num_classes = 7
epochs = 50

# Define directory
train_dir = './training_set'
test_dir = './validation_set'

In [7]:
# Create a data generator to load images from directories
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

Found 1031 images belonging to 7 classes.
Found 258 images belonging to 7 classes.


In [8]:
# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
# Train the model
history = model.fit(
        train_generator,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=validation_generator)

Epoch 1/50
33/33 [==============================] - 81s 2s/step - loss: 2.0293 - accuracy: 0.1455 - val_loss: 1.9459 - val_accuracy: 0.1512
Epoch 2/50
33/33 [==============================] - 64s 2s/step - loss: 1.9466 - accuracy: 0.1445 - val_loss: 1.9459 - val_accuracy: 0.1512
Epoch 3/50
33/33 [==============================] - 67s 2s/step - loss: 1.9467 - accuracy: 0.1503 - val_loss: 1.9451 - val_accuracy: 0.1550
Epoch 4/50
33/33 [==============================] - 64s 2s/step - loss: 1.9450 - accuracy: 0.1591 - val_loss: 1.9441 - val_accuracy: 0.1550
Epoch 5/50
33/33 [==============================] - 59s 2s/step - loss: 1.9238 - accuracy: 0.2076 - val_loss: 1.8450 - val_accuracy: 0.2791
Epoch 6/50
33/33 [==============================] - 66s 2s/step - loss: 1.6994 - accuracy: 0.3269 - val_loss: 1.6212 - val_accuracy: 0.3256
Epoch 7/50
33/33 [==============================] - 59s 2s/step - loss: 1.4663 - accuracy: 0.4239 - val_loss: 1.5040 - val_accuracy: 0.3837
Epoch 8/50
33/33 [==

In [ ]:
# Evaluate the model
score = model.evaluate(validation_generator, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

In [ ]:
# Save du model
model.save('my_model.h5')

TEST DU MODELE

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

model = load_model('my_model.h5')

In [26]:
%ls

data_test.csv  my_model.h5      README.md   training_data.csv  validation_set/
images/        predictions.csv  result.csv  training_set/


In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './validation_set',
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

Found 258 images belonging to 7 classes.


In [29]:
pred_probabilities = model.predict(test_generator)
pred_classes = np.argmax(pred_probabilities, axis=1)
class_names = list(test_generator.class_indices.keys())
true_classes = test_generator.classes
for i in range(10):
    print(f"Image {i}: Predicted class: {class_names[pred_classes[i]]}, True class: {class_names[true_classes[i]]}")

9/9 [==============================] - 7s 782ms/step
Image 0: Predicted class: disgusted, True class: angry
Image 1: Predicted class: angry, True class: angry
Image 2: Predicted class: disgusted, True class: angry
Image 3: Predicted class: angry, True class: angry
Image 4: Predicted class: disgusted, True class: angry
Image 5: Predicted class: angry, True class: angry
Image 6: Predicted class: angry, True class: angry
Image 7: Predicted class: angry, True class: angry
Image 8: Predicted class: angry, True class: angry
Image 9: Predicted class: neutral, True class: angry


CREATION DES GRAPHIQUES POUR VERIFIER LA PRECISION DU MODELE

In [ ]:
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Epoch 1/50
13/33 [==========>...................] - ETA: 33s - loss: 0.0090 - accuracy: 0.9976